In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch.nn as nn
import torch

In [ ]:
labour_hours = 8 # static over time
alpha = 1 # production elasticity of labour
target_inflation = 0.02
xi = 0 # fraction of tax revenue to redistribute

no_households = 100
no_firms = 10


In [ ]:
#A simple neural network architecture
class genetic_network(nn.Module):
    def __init__(self,inp,hidd,out):
        super().__init__()
        self.architecture= nn.Sequential(
        nn.Linear(inp,hidd),
        nn.ReLU(),
        nn.Linear(hidd,out),
    
        )
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.uniform_(m.weight, a=-3.0, b=3.0)   
    def forward(self,x):
        logits = self.architecture(x)
        return logits
    
#A function to get the weights of the nn
def gene(network):
    flat_params = torch.cat([p.view(-1) for p in network.parameters()])
    return flat_params

def set_gene(gene):
    model = genetic_network()
    pointer = 0
    for param in model.parameters():
        numel = param.numel()
        new_values = gene[pointer:pointer+numel].view_as(param)
        with torch.no_grad():
            param.copy_(new_values)
        pointer+=numel
    return model

In [ ]:
#Recombination strategies
def recombine_weighted(gene_a,gene_b,alpha):
    return (gene_a*alpha + (1-alpha)*gene_b,gene_b*alpha + (1-alpha)*gene_a)

#def 1 point crossover
def crossover(gene_a,gene_b):
    m = int(len(gene_a) / 2)
    child1 = torch.cat([gene_a[:m], gene_b[m:]], dim=0)
    child2 = torch.cat([gene_b[:m], gene_a[m:]], dim=0)
    return child1, child2
#Uniform crossover
def uniform_crossover(gene_a,gene_b):
    mask = torch.rand_like(gene_a,dtype=torch.float)<0.5
    child1 = torch.where(mask,gene_a,gene_b)
    child2 = torch.where(mask,gene_b,gene_a)
    return child1,child2 

#Random reset mutate
def random_reset(gene_a,prob):
    gene_a = gene_a.clone()
    for idx in range(gene_a.numel()):
        if(random.random()<prob):
            gene_a.view(-1)[idx] = random.uniform(-0.1,0.1)
    return gene_a

In [ ]:
class Household:
    def __init__(self,num_firms):
        self.savings = 0
        self.income = 0

        input_len = 6 + num_firms # savings, income, interest rate, tax rate, tax credit, prices,skill
        output_len = 1

        self.mind = genetic_network(input_len,32,output_len)

        self.reward = 0
        self.skill = random.uniform(0,1)
    def act(self,interest_rate,tax_rate_house,tax_credits,firms):
        prices = [firm.price for firm in firms]
        avg_price = torch.mean(prices)
        input_data = torch.tensor(
            [self.savings, self.income, interest_rate, tax_rate_house, tax_credits,self.skill],
            dtype=torch.float32
        )
        input_data = torch.cat([input_data, prices])
        output = self.mind(input_data)
        output = torch.clamp(output, min=0)
        scaled_output = output * (torch.mean(prices) / avg_price)
        self.goods_requested = scaled_output.item()
        return self.goods_requested

    def step(self):
       
    def comp_reward(self):
        gamma = 0.5  # risk aversion coefficient
        nu = 0 # weight on labour disutility
        mu = 1 # weight on savings utility

        def utility(x):
            # incase fp inaccuracies
            if x <= 0:
                return 0
            return (x ** (1 - gamma)) / (1 - gamma)

        self.consumption = sum(self.goods_received)
        hours = labour_hours if self.wage > 0 else 0

        # CURRENTLY DISCOUNTING CONSUMPTION UTILITY
        self.reward = 0 * utility(self.consumption) - nu * hours ** 2 + mu * (1 if self.savings > 0 else -1) * utility(abs(self.savings))
